In [1]:
!pip install scikit-fuzzy

Defaulting to user installation because normal site-packages is not writeable


In [2]:
import sys
project_path = os.getcwd().split("Code")[0]
sys.path.append(project_path)
import numpy as np
import networkx as nx
import skfuzzy as fuzz
from Code.notebook.graph.GraphConstructor import GraphConstructor

# Funzione per eseguire Fuzzy C-means (FCM)
def fuzzy_c_means(graph, num_clusters):
    """
    Esegue Fuzzy C-means sul grafo per raggruppare i nodi in base alla loro appartenenza a più cluster.
    
    Args:
        graph (networkx.Graph): Il grafo su cui eseguire il clustering.
        num_clusters (int): Numero di cluster desiderati.
    
    Returns:
        dict: Un dizionario con il grado di appartenenza di ciascun nodo a ciascun cluster.
    """
    # Estrai la matrice di adiacenza come feature
    adjacency_matrix = nx.to_numpy_array(graph)
    
    # Trasformiamo la matrice di adiacenza in un array di distanze (per usare FCM)
    distance_matrix = np.max(adjacency_matrix) - adjacency_matrix  # Invertiamo la matrice di adiacenza per ottenere una matrice di distanze
    
    # Esegui Fuzzy C-means sulla matrice di distanze
    cntr, u, _, _, _, _, _ = fuzz.cluster.cmeans(distance_matrix.T, num_clusters, 2, error=0.005, maxiter=1000)
    
    # Restituisci i gradi di appartenenza per ciascun nodo
    # u contiene i gradi di appartenenza, i nodi sono associati tramite l'ordine degli indici
    membership = {node: u[:, i].tolist() for i, node in enumerate(graph.nodes)}  # Modifica questa riga
    
    return membership, cntr

# Crea il grafo utilizzando GraphConstructor
graph_builder = GraphConstructor()
graph_builder.build_graph()
graph = graph_builder.graph

# Numero di cluster desiderati
num_clusters = 3

# Esegui il clustering con Fuzzy C-means
fcm_membership, fcm_centers = fuzzy_c_means(graph, num_clusters)

# Copia il grafo originale per assegnare i cluster
gr = graph.copy()

# Assegna l'appartenenza "maggioritaria" a ciascun nodo
for node, membership_values in fcm_membership.items():
    # Troviamo il cluster con il valore di appartenenza massimo
    dominant_cluster = np.argmax(membership_values)  # Ottieni l'indice del cluster con il valore massimo
    gr.nodes[node]['cluster'] = dominant_cluster  # Assegniamo il cluster maggioritario al nodo

# Funzione per sanitizzare attributi e chiavi
def sanitize_graph_attributes(graph):
    """
    Sanitizza le chiavi e gli attributi del grafo per renderli compatibili con il formato GML.
    """
    for node in graph.nodes:
        attrs = graph.nodes[node]
        sanitized_attrs = {}
        for key, value in attrs.items():
            new_key = str(key).replace(" ", "_").replace("-", "_")  # Rimpiazza spazi e trattini
            sanitized_attrs[new_key] = str(value) if not isinstance(value, str) else value
        graph.nodes[node].clear()
        graph.nodes[node].update(sanitized_attrs)
    
    for u, v, attrs in graph.edges(data=True):
        sanitized_attrs = {}
        for key, value in attrs.items():
            new_key = str(key).replace(" ", "_").replace("-", "_")
            sanitized_attrs[new_key] = str(value) if not isinstance(value, str) else value
        graph.edges[u, v].clear()
        graph.edges[u, v].update(sanitized_attrs)

# Applica la sanitizzazione al grafo
sanitize_graph_attributes(gr)

# Esporta il grafo in formato GML
output_path = "graph_with_fcm_clusters.gml"
nx.write_gml(gr, output_path)

print(f"Clustering con Fuzzy C-means completato! Grafo esportato in: {output_path}")


Clustering con Fuzzy C-means completato! Grafo esportato in: graph_with_fcm_clusters.gml
